In [1]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [3]:
# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

kGsRead = pd.read_csv('OmicsData_Brep.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [6]:
##temp
namexlsS = 'NoGF_100cells_24of72hrs_S.txt'
condsS0 = pd.read_csv(namexlsS,header=0,index_col=0,sep="\t")

In [7]:
condsS0

,0,Ribosome,p53inac,p53ac,MDM2,Wip1,ATMP,ATRac,MDM2product1,MDM2product2,MDM2product3,MDM2product4,MDM2product5,MDM2product6,MDM2product7,MDM2product8,MDM2product9,MDM2pro,Wip1product1,Wip1product2,Wip1product3,Wip1product4,Wip1product5,Wip1product6,Wip1product7,Wip1product8,Wip1product9,Wip1pro,BRCA2,MSH6,MGMT,damageDSB,damageSSB,ppAKT_MDM2,pMDM2,ARF,MDM4,p53ac_MDM4,ATMinac,ATRinac,...,pMEK_RasT_CRaf_BRaf,ERK_ppMEK,pERK_ppMEK,RSK_ppERK,pRSKnuc_MKP1,ppERKnuc_MKP1,cFos_pRSKnuc,cFos_ppERKnuc,RKIP_DAG_PKC,PIP3_PTEN,PIP3_AKT_PIP3_PDK1,PIP3_pAKT_mTORC2,GSK3b_ppAKT,bCATENIN_GSK3b,TSC2_ppAKT,TSC2_ppERK,RhebT_TSC,EIF4EBP1_mTORC1active,S6K_mTORC1active,FOXO_ppAKT,PI3K1_mTORC1active,pERK_MKP3,ppERK_MKP3,ppERKnuc_pMKP1,RasT_BRaf,RasT_BRaf_BRaf,MEK_RasT_BRaf_BRaf,pMEK_RasT_BRaf_BRaf,EIF4E,EIF4EBP1_EIF4E,RasT_CRaf_CRaf,MEK_RasT_CRaf_CRaf,pMEK_RasT_CRaf_CRaf,FOXOnuc,MEKi,MEKi_ppMEK,AKTi,AKTi_AKT,mT,EIF4E_mT
0,8641,1899.795446,484.520360,4.934868,246.182604,1.031957,2.969825e-79,6.056783e-110,4.931557,4.925913,4.925569,4.897349,4.896226,4.893691,4.893344,4.887062,4.879934,4.871846,4.927095,4.915465,4.900758,4.892479,4.882576,4.867362,4.862344,4.859401,4.854776,4.853035,0.993486,45.698410,157.679594,0.000166,0.000005,0,0,0,0.025103,0,1.832747,3.346764,...,0.017543,1.692349,0.121018,0.341909,0.000002,0.000009,0.002073,0.008669,0,0.003388,0.000447,0.000360,0.014183,0,0.000455,0.000696,0,0.000827,0.002843,0.000575,0.000291,0.000037,0,0.000035,0.080937,0.000655,0.000805,0.000084,237.002902,15.603488,0.084396,0.570249,0.059736,10.558714,0,0,0,0,37.537147,88.962068
1,6316,1896.441433,480.575653,7.307207,166.096861,1.359976,1.115591e-73,5.732751e-108,7.303749,7.297377,7.296986,7.243188,7.241007,7.235860,7.235151,7.220602,7.200936,7.172248,7.298558,7.283290,7.258644,7.242376,7.217213,7.151210,7.124838,7.107557,7.076568,7.064412,0.491366,48.973639,112.127591,0.000315,0.000006,0,0,0,0.027665,0,1.437013,2.972431,...,0.021026,1.890031,0.157712,0.447593,0.000002,0.000009,0.002093,0.008713,0,0.003386,0.000346,0.000277,0.010718,0,0.000464,0.001204,0,0.000166,0.000417,0.000381,0.000047,0.000030,0,0.000046,0.087285,0.000762,0.000942,0.000107,229.160777,17.318866,0.088237,0.599969,0.067882,9.098160,0,0,0,0,38.431243,88.067973
2,8641,1896.236869,317.633322,6.850961,181.103376,2.514060,3.981740e-79,2.447584e-107,6.848730,6.844369,6.844101,6.821444,6.820542,6.818519,6.818242,6.813482,6.808536,6.803607,6.845177,6.835342,6.823143,6.816653,6.809715,6.801339,6.798884,6.797547,6.795604,6.794888,1.393569,45.311406,104.805063,0.000167,0.000007,0,0,0,0.022609,0,1.628828,2.722759,...,0.030129,2.528597,0.270328,0.820728,0.000003,0.000011,0.007256,0.028235,0,0.003407,0.000403,0.000361,0.013493,0,0.000656,0.002235,0,0.000076,0.000232,0.000632,0.000035,0.000041,0,0.000104,0.117987,0.001392,0.001442,0.000203,240.335262,13.517966,0.091222,0.519853,0.073180,11.570193,0,0,0,0,37.169518,89.329697
3,8641,1896.282510,354.121932,6.503964,197.464162,0.977220,1.413434e-68,1.036782e-108,6.504109,6.504843,6.504889,6.528523,6.529501,6.531910,6.532244,6.539896,6.551623,6.571223,6.504816,6.508110,6.516955,6.524114,6.538040,6.586516,6.607167,6.621154,6.647391,6.657839,0.480625,33.988473,141.492112,0.000564,0.000006,0,0,0,0.019076,0,1.398768,2.185850,...,0.017148,1.901881,0.144063,0.423268,0.000002,0.000008,0.001087,0.004371,0,0.003379,0.000430,0.000364,0.016338,0,0.000554,0.000997,0,0.000109,0.000375,0.000453,0.000042,0.000040,0,0.000041,0.071315,0.000509,0.000632,0.000068,249.420884,16.016816,0.096759,0.661223,0.070955,8.198760,0,0,0,0,36.202998,90.296217
4,8641,1896.540656,299.475748,6.462645,176.872903,4.203688,4.972520e-71,3.833931e-108,6.456070,6.444927,6.444247,6.389046,6.386850,6.381892,6.381213,6.368831,6.354435,6.337406,6.447300,6.424814,6.396468,6.380274,6.359891,6.326412,6.315274,6.308796,6.299361,6.295955,0.100428,44.998515,119.939324,0.000430,0.000006,0,0,0,0.018807,0,1.541794,1.986955,...,0.011523,1.306600,0.068739,0.202000,0.000001,0.000005,0.001856,0.007463,0,0.003407,0.00035

In [4]:
flagD = 0
th = 30
numStocCells = 30
Meki_ind = 768
Akti_ind = 770
inh_conc = 10000.0
STIMligs = [10.0,0.0,0.0,0.0,0.0,0.0,0.0]
nmxlsfile1 = 'Fig6B_MAi_v1_25cells_30hrs_' # E # THIS CONDITION WILL BE SIMULATED IN THIS NOTEBOOK
nmxlsfile2 = 'Fig6B_MAi_v2_25cells_30hrs_' # I
nmxlsfile3 = 'Fig6B_MAi_v3_25cells_30hrs_' # E+I
nmxlsfile4 = 'Fig6B_MAi_v4_25cells_30hrs_' # E+I+Meki
nmxlsfile5 = 'Fig6B_MAi_v5_25cells_30hrs_' # E+I+Akti

ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)
Vn = 1.75E-12
Vc = 5.25E-12
tout_all = np.arange(0,th*3600+1,30)

genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)
sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints
model0 = model
currentDirectory = os.getcwd()

# Use pre-simulated stochastic cells without any ligands (24 hours, serum-starvation cond). 
# The end-point of each pre-simulated cell is the starting condition for one cell
namexlsS = 'NoGF_100cells_24of72hrs_S.txt'
condsS0 = pd.read_csv(namexlsS,header=0,index_col=0,sep="\t")
spdataMat0 = condsS0.loc[:,1:]
namexlsG = 'NoGF_100cells_24of72hrs_G.txt'
condsG0 = pd.read_csv(namexlsG,header=0,index_col=0,sep="\t")
genedataMat0 = condsG0.loc[:,1:]

startTime = datetime.now()
print(startTime)
for nn in range(numStocCells):
    # E+I+Meki+Akti
    spdata = np.array(spdataMat0.loc[nn,:])
    spdata[155:162] = STIMligs
#     spdata[Meki_ind] = inh_conc
#     spdata[Akti_ind] = inh_conc
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata # 24hr time point     
    
    genedata = np.array(genedataMat0.loc[nn,:])
    xoutG_all = np.zeros(shape=(NSteps+1,len(genedata)))
    xoutG_all[0,:] = genedata
    model2 = model0
    solver2 = model2.getSolver() # Create solver instance
    solver2.setMaxSteps = 1e10
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model2.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model2.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model2, solver2)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            break
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model2.getStateIds()]) 
    condsSDF.to_csv(nmxlsfile1+'S_'+str(nn)+'.txt',sep="\t")
    condsSDF = None
    condsGDF = pd.DataFrame(data=xoutG_all) 
    condsGDF.to_csv(nmxlsfile1+'G_'+str(nn)+'.txt',sep="\t")    
    condsGDF = None    
print(datetime.now())

RuntimeError: Cannot use model SPARCED_Brep, generated with AMICI version 0.10.13, together with AMICI version 0.11.16 which is present in your PYTHONPATH. Install the AMICI package matching the model version or regenerate the model with the AMICI currently in your path.

In [ ]:
numStocCells = 30
ttds = []
cellsinsphase = np.zeros(shape=(numStocCells,1))
deadcells = np.zeros(shape=(numStocCells,1))
currentDirectory = os.getcwd()
nmxlsfile = 'Fig6B_MAi_v1_25cells_30hrs_S_'
plotind = 58
suminds = [49,58,68]
plt.figure(figsize=(7, 4))
for cc in range(numStocCells):
    namer = currentDirectory+'\\Fig6B\\'+nmxlsfile+str(cc)+'.txt'
    condsDF = pd.read_csv(namer,header=0,index_col=0,sep="\t")
    condsDF = np.double(condsDF.values[:])
    condsDF = condsDF[~np.all(condsDF == 0, axis=1)]
    yy = np.array(condsDF[:,plotind])
    tt = (np.arange(0,len(condsDF))*30.0/3600.0)
    plt.plot(tt, yy,linewidth=4)
    th24ind = np.argwhere(tt==24.0)
    if len(th24ind)>0:
        ss = np.sum(condsDF[th24ind,suminds])
        if ss>20.0:
            cellsinsphase[cc] = 1
    else:
        deadcells[cc] = 1
    condsDF = None
plt.ylim([0, 50])
plt.xlim([0, 30])
plt.savefig('Fig6B_v1.png')